In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px

In [65]:
# 讀取ZMAW原始數據
df = pd.read_clipboard()
df.columns = df.columns.str.strip()
df["實際出貨日"] = pd.to_datetime(df["實際出貨日"])

In [2]:
# 讀取簽約客戶客編對照表
cus_df = pd.read_clipboard()
cus_code = cus_df["客編"].str.replace("[", "").str.replace("]", "").str.replace("'", "").str.replace(" ", "").str.split(",", expand=True)
cus_df["整理的代號"] = cus_code.fillna("").values.tolist()
cus_df = cus_df.explode("整理的代號")
cus_df = cus_df[cus_df["整理的代號"] != ""]
cus_dict = dict(zip(cus_df["整理的代號"], cus_df["客戶名稱"]))

In [57]:
# 取出有效訂單
df = df[df["訂單類型"].isin(["ZSO1", "ZKE", "ZOR1"])]

In [60]:
# 將客戶客編對照表map進實績客戶名稱
df["簽約客戶"] = df["買方"].map(cus_dict)

In [67]:
# 客戶通路dict讀取
channel = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\ReferenceTable\20220801_category_types.xlsx")
ch_dict = dict(zip(channel["客戶代號"], channel["大通路"]))

In [82]:
# map通路表到實績通路客戶編號
df["大通路"] = df["買方"].map(ch_dict)

In [84]:
# 建立欄位檢視是否為簽約客戶
df["是否簽約"] = df["簽約客戶"].map(lambda x: "Yes" if pd.notna(x) else "No")

In [85]:
df.to_clipboard()

In [91]:
# 製圖的groupby
result = df.groupby(["簽約客戶"])[["銷售數量"]].sum().reset_index()

In [160]:
result.describe()

,銷售數量,月均條數
count,67.00000,67.000000
mean,207.58209,69.194030
std,225.06081,75.040441
min,15.00000,5.000000
25%,73.50000,24.500000
50%,131.00000,44.000000
75%,239.00000,79.500000
max,1158.00000,386.000000


In [135]:
result["月均條數"] = round(result["銷售數量"] / 3, 0)

In [136]:
px.box(result, y="月均條數", title="1-3月簽約客戶月均條數箱型圖")

In [150]:
bins = [0, 30, 60, 125, 1000]
labels = ["0-29", "30-59", "60-124", "125以上"]
result["分組"] = pd.cut(result["月均條數"], bins, labels=labels, right=False)

In [151]:
result

,簽約客戶,銷售數量,分組,月均條數
0,三和輪業有限公司,131,30-59,44.0
1,世大日實業社,595,125以上,198.0
2,中華全佑有限公司,86,0-29,29.0
3,五七輪胎行,148,30-59,49.0
4,信興輪胎行,151,30-59,50.0
...,...,...,...,...
62,順發輪胎行,130,30-59,43.0
63,駿濠實業有限公司,221,60-124,74.0
64,麒駪,70,0-29,23.0
65,鼎鑫輪胎行,34,0-29,11.0


In [155]:
px.histogram(result, x="分組", text_auto=True).update_layout(font=dict(size=20))

In [153]:
px.histogram(result, x="簽約客戶", y="月均條數", title="1-3月簽約客戶月均條").update_layout(xaxis = {"categoryorder": "total descending"}).add_hline(y=result["銷售數量"].mean(), line_width=3, line_dash="dash", line_color="red")

In [157]:
result["等級"] = result["分組"].map(lambda x: "S" if x == "125以上" else "A" if x == "60-124" else "B" if x == "30-59" else "未達門檻")

In [159]:
result.to_clipboard()